# Baseline models
Several models were designed as baseline: 
- Copy-Paste: Copy the latest data
- Markovian: Use all of the data from the previous day
- Non-Markovian: Use all the data up to the decorrelation time

In [1]:
#import useful libraries
import importlib
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from scipy import sparse
stf_dir = os.path.join(os.pardir, 'spacetimeformer')
sys.path.append(stf_dir)
#from spacetimeformer import data
#importlib.reload(data) #to make sure the last version of stf is used

In [2]:
#delete if not used by Thursday
class forecaster():
    def __init__(self,model,dset,target,lag):
        assert(model in ['CP','Mark','NMark','LSTM'])
        assert(dset in ['full','selected'])
        assert(target in ['chla','clusters','chcy'])
                
        
    def fit(self,x_t,y_t):
        return
    def score(self,x_v,y_v):
        return
    
            
forecaster('CP','selected','chla',1)

In [3]:
#load the dataset
data_path = "..\\Datasets\\Forecasting_aqua\\data_h_rolling_cst_int_v4.csv"
format = "%Y-%m-%d %H:%M:%S"
df = pd.read_csv(data_path)
time_df = pd.to_datetime(df["Datetime"], format=format)
try:
    df.drop("sample",axis=1,inplace=True)
except:
    pass
df['Datetime']= time_df
#According to the work of Stefanie Merkly, these keys were more important than the others to predict chorophill a
keys_imp = ['ciliate','mean_chla','cv_chla_day','cv_chla_depth',
            'nauplius','Ammonium','Nitrat','mean_schmidt','windspeed_max',
            'mean_thermocline_depth','mean_epi_temp','mean_oxycline_depth',
            'mean_mixed_layer_depth','mean_global_radiation'
           ]
#add some usefull variables at the end of the table
df['year'] = df['Datetime'].dt.year
df['month'] = df['Datetime'].dt.month
df['day'] = df['Datetime'].dt.day

#drop the first datapoints the first months are only meteorological data
first_point = (~df['diatom'].isnull()).idxmax()
dfs = df.drop(index=range(first_point))
df = dfs.reset_index(drop=True)

In [4]:
def score(y_true,y_pred):
    R2 = 1-np.sum(np.square(y_true-y_pred))/np.sum(np.square(y_true-np.mean(y_true,axis=0)))
    MAEn = 1-np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true-np.mean(y_true,axis=0)))
    return {'R2':R2,'MAEn':MAEn}

In [100]:
def kfold(model,input_tv,n_fold,input_test,target_tv,target_test):
    breakpoints = np.linspace(0,len(input_tv),n_fold+1,dtype=int)
    res = []
    for fold in range(n_fold):
        print(f"Fitting fold n°{fold}")
        input_v = input_tv[breakpoints[fold]:breakpoints[fold+1]]
        y_v  = target_tv[breakpoints[fold]:breakpoints[fold+1]]
        input_t = pd.concat([input_tv[:breakpoints[fold]],input_tv[breakpoints[fold+1]:]])
        y_tr = pd.concat([target_tv[:breakpoints[fold]],target_tv[breakpoints[fold+1]:]])
        model.fit(input_t,y_tr,[breakpoints[fold],breakpoints[fold+1]])
        y_predtr = model.pred(input_t)
        y_predv = model.pred(input_v)
        y_predte = model.pred(input_test)
        
        s_train = score(y_tr.to_numpy(),y_predtr)
        s_val = score(y_v.to_numpy(),y_predv)
        s_test = score(target_test.to_numpy(),y_predte)
        res.append({'fold':fold,'bp': breakpoints, 'y_predv':y_predv,'s_train':s_train,'s_val':s_val,'s_test':s_test})
    return res

In [99]:
class CP_model:
    def __init__(self,target_keys:list):
        self.targets = target_keys
    def fit(self,inputs,targets,breakpoints):
        return
    def pred(self,inputs):
        return inputs[self.targets].to_numpy()
class Markov_model:
    def __init__(self,target_keys:list, inputs_keys:list,norm=True):
        from sklearn.linear_model import LinearRegression
        from sklearn.preprocessing import StandardScaler
        self.skmodel = LinearRegression()
        if norm:
            self._scalerin = StandardScaler()
            self._scalerout = StandardScaler()
    def fit(self,inputs,target,breakpoints):
        inputs = inputs.drop(columns=['Datetime'],errors='ignore')
        self._scalerin = self._scalerin.fit(inputs.values)
        self._scalerout = self._scalerout.fit(target.values)
        ninp = self._scalerin.transform(inputs.values)
        #fill nans with 0s
        ninp = np.nan_to_num(ninp)
        ntar = self._scalerout.transform(target.values)
        self.skmodel.fit(ninp,ntar)
    def pred(self,inputs):
        inputs = inputs.drop(columns=['Datetime'],errors='ignore')
        ninp = self._scalerin.transform(inputs.values)
        ninp = np.nan_to_num(ninp)
        npred = self.skmodel.predict(ninp)
        return self._scalerout.inverse_transform(npred)
    

class NONMarkov_model:
    def __init__(self,target_keys:list, inputs_keys:list,norm=True,cor_th = 0.2,
                 max_lag = 60*24,decimate = 10):
        from sklearn.linear_model import LinearRegression
        from sklearn.preprocessing import StandardScaler
        #get the decorrelation time
        self.skmodel = [LinearRegression() for dim in range(len(target_keys))]
        self.target_keys = target_keys
        if norm:
            self._scalerin = StandardScaler()
            self._scalerout = StandardScaler()
        self.cor_th = cor_th
        self.max_lag = max_lag
        self.decimate = decimate
        self.cor_masks = []
    def fit(self,inputs,targets,breakpoints):
        inputs = inputs.drop(columns=['Datetime'],errors='ignore')
        #fill the validation fold with 0:
        self._scalerin = self._scalerin.fit(inputs.values)
        self._scalerout = self._scalerout.fit(targets.values)
        ninp = self._scalerin.transform(inputs.values)
        #fill nans with 0s
        ninp = np.nan_to_num(ninp)
        ntar = self._scalerout.transform(targets.values)
        ntar = np.nan_to_num(ntar)
        #process each out key separatly:
        ninp_pad = np.concatenate([ninp[:breakpoints[0]],
                                   np.zeros((breakpoints[1]-breakpoints[0],ninp.shape[1])),
                                   ninp[breakpoints[1]:]])
        ntar_pad = np.concatenate([ntar[:breakpoints[0]],
                                   np.zeros((breakpoints[1]-breakpoints[0],ntar.shape[1])),
                                   ntar[breakpoints[1]:]])

        for target in range(ntar.shape[1]):
            cor = signal.correlate(ninp_pad,np.expand_dims(ntar_pad[:,target],-1), mode='full')/len(ntar_pad)
            #note: due to the empty space left by the validation set, the correlation cannot reach one
            cor_lag = signal.correlation_lags(ninp_pad.shape[0],ntar_pad.shape[0], mode='full')
            idx0 = np.where(cor_lag==0)[0][0]
            cor_past = cor[:idx0]
            cor_mask = cor_past>self.cor_th
            longest_lag = np.argmax(np.any(cor_mask,axis=1))
            max_lag = min(longest_lag,self.max_lag)
            #cor_lag = cor_lag[max_lag:idx0-24*lag]
            cor_mask = cor_mask[idx0-max_lag:idx0]
            self.cor_masks.append(cor_mask)

            #format the input
            start_inp = np.concatenate([np.arange(0,breakpoints[0]-max_lag),
                                        np.arange(breakpoints[1],ninp.shape[0]-max_lag)])
            inp_nonmarkov = np.zeros((len(start_inp)//self.decimate+1,max_lag*ninp.shape[1]))
            tar_nonmarkov = np.zeros(len(start_inp)//self.decimate+1)
            for idx,start in enumerate(start_inp[::self.decimate]):
                inp_nonmarkov[idx,:] = (cor_mask * ninp[start:start+max_lag]).flatten()
                tar_nonmarkov[idx] = ntar[start+max_lag]
            #fit the model:
            print("inputs formated")
            model = self.skmodel[target].fit(inp_nonmarkov,tar_nonmarkov)
            print("model trained")
    def pred(self,inputs):
        inputs = inputs.drop(columns=['Datetime'],errors='ignore')
        ninp = self._scalerin.transform(inputs.values)
        ninp = np.nan_to_num(ninp)
        pred_nonmarkov = np.empty((len(inputs),len(self.target_keys)))
        pred_nonmarkov[:] = np.nan
                                  
        #format:
        for target in len(self.target_keys):
            cor_mask = self.cor_masks[target]
            #format the input
            start_inp = np.arange(0,len(ninp)-max_lag)
            inp_nonmarkov = np.zeros((len(start_inp),max_lag*ninp.shape[1]))
            
            for idx,start in enumerate(start_inp):
                inp_nonmarkov[idx,:] = (cor_mask * ninp[start:start+max_lag,:]).flatten()
            pred_nonmarkov[-len(start_inp):,target] = self.skmodel[target].predict(inp_nonmarkov)
        return self._scalerout.inverse_transform(npred)

In [88]:
#init:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
scalerin = StandardScaler()
scalerout = StandardScaler()

In [93]:
i = 3000
breakpoints = [i,i+2194]
inputs = inputs_tv.set_index('Datetime')
target = targets_tv['mean_chla_depth']
scalerin = scalerin.fit(inputs.values)
scalerout = scalerout.fit(target.values.reshape(-1, 1))
ninp = scalerin.transform(inputs.values)
ninp = np.nan_to_num(ninp)
ntar = scalerout.transform(target.values.reshape(-1, 1))
ntar = np.nan_to_num(ntar)
ninp_pad = np.concatenate([ninp[:breakpoints[0]],np.zeros((breakpoints[1]-breakpoints[0],ninp.shape[1])),ninp[breakpoints[1]:]])
ntar_pad = np.concatenate([ntar[:breakpoints[0]],np.zeros((breakpoints[1]-breakpoints[0],ntar.shape[1])),ntar[breakpoints[1]:]])

cor = signal.correlate(ninp_pad,np.expand_dims(ntar_pad[:,0],-1), mode='full')/len(ntar_pad)
#note: due to the empty space left by the validation set, the correlation cannot reach one

cor_lag = signal.correlation_lags(ninp_pad.shape[0],ntar_pad.shape[0], mode='full')
idx0 = np.where(cor_lag==0)[0][0]
cor_past = cor[:idx0]
cor_mask = cor_past>0.2
longest_lag = np.argmax(np.any(cor_mask,axis=1))
max_lag = min(longest_lag,24*50)
#cor_lag = cor_lag[max_lag:idx0-24*lag]
cor_mask = cor_mask[idx0-max_lag:idx0]

#format the input
decimate =5
start_inp = np.concatenate([np.arange(0,breakpoints[0]-max_lag),np.arange(breakpoints[1],ninp.shape[0]-max_lag)])
inp_nonmarkov = np.zeros((len(start_inp)//decimate+1,max_lag*ninp.shape[1]))
tar_nonmarkov = np.zeros(len(start_inp)//decimate+1)
for idx,start in enumerate(start_inp[::decimate]):
    inp_nonmarkov[idx,:] = (cor_mask * ninp[start:start+max_lag]).flatten()
    
    tar_nonmarkov[idx] = ntar[start+max_lag]
#fit the model:
print("inputs formated")
model = LinearRegression().fit(inp_nonmarkov,tar_nonmarkov)
print("model trained")

inputs formated
model trained


In [105]:
#model = CP_model(['mean_chla_depth'])
model = NONMarkov_model(['mean_chla_depth','cluster_1'],df.keys())

In [106]:
test_prop = 0.1
n_fold = 10
lag = 1
in_keys = df.keys()
out_keys = ['mean_chla_depth','cluster_1']
test_len = int(len(df)*test_prop)
dftest = df[-test_len:].reset_index(drop=True)
dftv = df[:-test_len]

inputs_tv = dftv[in_keys][:-24*lag]
targets_tv = dftv[out_keys][24*lag:].reset_index(drop=True)

inputs_test = dftest[in_keys][:-24*lag]
targets_test = dftest[out_keys][24*lag:].reset_index(drop=True)


In [ ]:
res = kfold(model,inputs_tv,n_fold,inputs_test,targets_tv,targets_test)


In [ ]:
res = kfold(model,inputs_tv,n_fold,inputs_test,targets_tv,targets_test)


Fitting fold n°10


In [ ]:
def nice_plot(inputs_tv,inputs_test,targets_tv,targets_test,res):
    fig,axs = plt.subplots(len(targets_tv.keys()),figsize = (15,6*len(targets_tv.keys())))
    if len(targets_tv.keys()):
        axs=[axs]
    for idx,(ax,key) in enumerate(zip(axs,targets_tv.keys())):
        val_concat = np.concatenate([f['y_predv'] for f in res])
        true = ax.scatter(inputs_tv['Datetime'],targets_tv.to_numpy(),s=1)
        for fold in range(len(res[0]['bp'])-1):
            pred = ax.scatter(inputs_tv['Datetime'][res[0]['bp'][fold]:res[0]['bp'][fold+1]],res[fold]['y_predv'],s=0.5)
def plot_res(res):
    print(f"R2 val: {np.mean([f['s_val']['R2'] for f in res]):.2} +/- {2*np.std([f['s_val']['R2'] for f in res]):.2}")
    print(f"R2 test: {np.mean([f['s_test']['R2'] for f in res]):.2} +/- {2*np.std([f['s_val']['R2'] for f in res]):.2}")
    

In [ ]:
nice_plot(inputs_tv,inputs_test,targets_tv,targets_test,res)

In [ ]:
plot_res(res)